In [1]:
import os
import json
from zipfile import ZipFile

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
kaggle_ditionary = json.load(open('kaggle.json'))

In [3]:
os.environ['KAGGLE_USERNAME'] = kaggle_ditionary['username']
os.environ['KAGGLE_KEY'] = kaggle_ditionary['key']

In [4]:
data = pd.read_csv('imdb-dataset-of-50k-movie-reviews\\IMDB Dataset.csv')

In [5]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
data.replace({"sentiment" : {"positive" : 1, "negative" : 0}}, inplace=True)
train_data,test_data = train_test_split(data, test_size=0.2, random_state=42)

C:\Users\agent\AppData\Local\Temp\ipykernel_12340\2520469502.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment" : {"positive" : 1, "negative" : 0}}, inplace=True)


In [7]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen = 200)

In [8]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [9]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None, 200))

c:\Users\agent\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 161ms/step - accuracy: 0.7384 - loss: 0.5133 - val_accuracy: 0.8451 - val_loss: 0.3823
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 92s 184ms/step - accuracy: 0.8323 - loss: 0.3867 - val_accuracy: 0.8409 - val_loss: 0.3634
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 196ms/step - accuracy: 0.8744 - loss: 0.3092 - val_accuracy: 0.8522 - val_loss: 0.3384
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 168ms/step - accuracy: 0.8837 - loss: 0.2902 - val_accuracy: 0.8608 - val_loss: 0.3442
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 83s 166ms/step - accuracy: 0.8967 - loss: 0.2628 - val_accuracy: 0.8721 - val_loss: 0.3306


In [12]:
loss, accuracy = model.evaluate(X_test,Y_test)
print(f"Test loss : {loss} Test accuracy : {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8731 - loss: 0.3197
Test loss : 0.3203740119934082 Test accuracy : 0.8743000030517578


In [13]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen = 200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

In [ ]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
The sentiment of the review is : positive


In [ ]:
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
The sentiment of the review is: negative


In [14]:
import pickle

# Save the trained model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)
with open('tokenizer.pkl','wb') as file:
    pickle.dump(tokenizer, file)